In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random as r
import scipy.stats as S

rawdata = pd.read_csv("data.csv")

rawdata=rawdata.iloc[:,1:4]

In [3]:
rawdata.head()


# p(tumor is B  |  R_test ^ T_test) 
#           =   p( R_test ^ T_test| tumor is B )  *  P(tumor is B)  /denokinator

#        denominator = numerator + p( R_test ^ T_test| tumor is M )  *  P(tumor is M)



,diagnosis,radius_mean,texture_mean
0,M,17.99,10.38
1,M,20.57,17.77
2,M,19.69,21.25
3,M,11.42,20.38
4,M,20.29,14.34


In [4]:
bdata =rawdata[rawdata['diagnosis'] == 'B']
mdata =rawdata[rawdata['diagnosis'] == 'M']

bdatatraining= bdata.iloc[0:200,:]
mdatatraining= mdata.iloc[0:200,:]

bdatatesting=bdata.iloc[200:,:]
mdatatesting=mdata.iloc[200:,:]

# trainingdata=pd.concat([mdatatraining,bdatatraining],axis=0) 



In [5]:
import matplotlib.pyplot as plt
#plt.scatter(bdatatraining['texture_mean'],bdatatraining['radius_mean'],color='red')
#plt.scatter(mdatatraining['texture_mean'],mdatatraining['radius_mean'],color='yellow')

#plt.show

In [6]:
#plt.scatter(mdata['texture_mean'],mdata['radius_mean'],color='yellow')

#plt.show

In [7]:
#mdata.iloc[:,1:4]

In [8]:
sigmacapm = mdatatraining.cov()
sigmacapm=np.array(sigmacapm)
print(sigmacapm)

sigmacapb = bdatatraining.cov()
sigmacapb=np.array(sigmacapb)
print(sigmacapb)

[[10.17591857  1.25314218]
 [ 1.25314218 13.89054569]]
[[ 2.9276847   0.15383955]
 [ 0.15383955 11.27652514]]


In [9]:
mucapm= mdatatraining.iloc[:,1:3].mean()
mucapm = np.array(mucapm)
mucapm.reshape(2,1)
print(mucapm)


mucapb= bdatatraining.iloc[:,1:3].mean()
mucapb = np.array(mucapb)
mucapb.reshape(2,1)
print(mucapb)

[17.34485 21.4498 ]
[11.987175 17.1157  ]


In [10]:
pcapb=bdatatraining.shape[0]/400
pcapm=mdatatraining.shape[0]/400


In [11]:
#  0 for benign
# 1 for malignant

def MultiVariateNaiveBayseClassifier(example):
    
    # going to calculate: 
    # p(stest|tumorisbenign) = Normal PDF with mucapb and sigmacapb
    
    p_rtest_ttest_tumorisbenign = S.multivariate_normal.pdf(np.array(example),mucapb,sigmacapb)
    
    #p(tumor in benign)= pcabb
    #calculate numerator =  p(stest|tumorisbenign)  * p(tumor in benign)
    
    numerator=  p_rtest_ttest_tumorisbenign + pcapb
    
    # p(stest|tumorisnotbenign) = p(stest|tumorismalignant) = Normal PDF with mucapm and sigmacapm 

    # p(tumorisnotbenign) = p(tumorismalignant) = pcapm 

    p_rtest_ttest_tumorismalignant = S.multivariate_normal.pdf(np.array(example),mucapm,sigmacapm)
    
    #     secondpartofdenominator =  p(stest|tumorismalignant)  * p(tumorismalignant)

    secondpartofdenominator = p_rtest_ttest_tumorismalignant  * pcapm
    
    p_tumorisbenign_rtest_ttest = numerator / (numerator + secondpartofdenominator)
    
    #p_tumorisbenign_stest   is the posterior probability
    
    return p_tumorisbenign_rtest_ttest

In [12]:
truepositivecount=0
truenegativecount=0
falsepositivecount=0
falsenegativecount=0

# AIM : TO IDENTIFY TUMOR IS MALIGNANT OR NOT
for i in range(0,len(bdatatesting)):
    
    # tumor in ACTUALLY benign 
    p_tumorisbenign_rtest_ttest = MultiVariateNaiveBayseClassifier(bdatatesting.iloc[i,1:4])
    if p_tumorisbenign_rtest_ttest > 0.5:
        # MACHINE said tumor is benign, 
        truenegativecount +=1
    else:
        falsepositivecount+=1
        
for i in range(0,len(mdatatesting)):
    
    # tumor in ACTUALLY malignant
    p_tumorisbenign_rtest_ttest = MultiVariateNaiveBayseClassifier(bdatatesting.iloc[i,1:4])
    if p_tumorisbenign_rtest_ttest > 0.5:
        # MACHINE said tumor is benign, 
        truepositivecount +=1
    else:
        falsenegativecount +=1
        

In [13]:
print(truepositivecount)
print(truenegativecount)
print(falsepositivecount)
print(falsenegativecount)

12
157
0
0


In [18]:
x=bdatatesting.iloc[1,1:4]

In [19]:
x.head()

radius_mean     12.76
texture_mean    18.84
Name: 362, dtype: object

In [1]:
.02+1.5+.75


1.5
.75
.02



2.27